# GameTheory 8 - Jeux Combinatoires

**Navigation** : [<< 7-ExtensiveForm](GameTheory-7-ExtensiveForm.ipynb) | [Index](README.md) | [9-BackwardInduction >>](GameTheory-9-BackwardInduction.ipynb)

**Side tracks** : [8b-Lean-CombinatorialGames](GameTheory-8b-Lean-CombinatorialGames.ipynb) | [8c-CombinatorialGames-Python](GameTheory-8c-CombinatorialGames-Python.ipynb)

**Kernel** : Python 3

---

## Introduction

Les **jeux combinatoires** sont une classe speciale de jeux :
- **2 joueurs** (Gauche et Droite)
- **Alternance** des coups
- **Information parfaite** (pas de hasard, pas de coups caches)
- **Terminaison** : le jeu finit toujours
- **Normal play** : le joueur qui ne peut plus jouer perd

Contrairement aux jeux strategiques (Nash, equilibres mixtes), les jeux combinatoires n'ont pas d'utilites numeriques - seulement **victoire ou defaite**.

### Jeux impartiaux vs partizans

| Type | Coups disponibles | Exemples |
|------|------------------|----------|
| **Impartial** | Memes pour les deux joueurs | Nim, Jeux de soustraction |
| **Partizan** | Differents selon le joueur | Echecs, Go, Domineering |

Dans ce notebook, nous nous concentrons sur les **jeux impartiaux** et le celebre **theoreme de Sprague-Grundy**.

### Plan

1. Positions P et N
2. Le jeu de Nim
3. Fonction Mex et valeur de Grundy
4. Theoreme de Sprague-Grundy
5. Jeux de soustraction
6. Exercices

---

In [1]:
# Configuration
import numpy as np
import matplotlib.pyplot as plt

print("Notebook 8 - Jeux Combinatoires")
print("================================")

Notebook 8 - Jeux Combinatoires


---

## 1. Positions P et N

Dans un jeu combinatoire, chaque position est soit :

- **P-position** (Previous player wins) : Le joueur qui vient de jouer gagne
- **N-position** (Next player wins) : Le joueur qui va jouer gagne

### Regles de classification

1. Une position **terminale** (sans coups possibles) est une **P-position**
2. Une position est une **N-position** si elle a au moins un coup vers une P-position
3. Une position est une **P-position** si tous ses coups menent a des N-positions

### Exemple : Jeu de 1-2-3

Regles : Un tas de n jetons, on peut retirer 1, 2 ou 3 jetons. Qui retire le dernier gagne.

In [2]:
def classify_123_game(n_max: int) -> dict:
    """
    Classifie les positions du jeu 1-2-3.
    Retourne un dict {position: 'P' ou 'N'}
    """
    positions = {}
    
    for n in range(n_max + 1):
        if n == 0:
            # Position terminale : le joueur precedent a pris le dernier jeton
            positions[n] = 'P'
        else:
            # Verifier si on peut atteindre une P-position
            can_reach_P = False
            for take in [1, 2, 3]:
                if n >= take and positions.get(n - take) == 'P':
                    can_reach_P = True
                    break
            positions[n] = 'N' if can_reach_P else 'P'
    
    return positions

# Classifier les 16 premieres positions
positions = classify_123_game(16)

print("Position | Type | Motif")
print("-" * 25)
for n in range(17):
    t = positions[n]
    motif = "Perdante (n % 4 == 0)" if t == 'P' else "Gagnante"
    print(f"   {n:2d}    |  {t}   | {motif}")

Position | Type | Motif
-------------------------
    0    |  P   | Perdante (n % 4 == 0)
    1    |  N   | Gagnante
    2    |  N   | Gagnante
    3    |  N   | Gagnante
    4    |  P   | Perdante (n % 4 == 0)
    5    |  N   | Gagnante
    6    |  N   | Gagnante
    7    |  N   | Gagnante
    8    |  P   | Perdante (n % 4 == 0)
    9    |  N   | Gagnante
   10    |  N   | Gagnante
   11    |  N   | Gagnante
   12    |  P   | Perdante (n % 4 == 0)
   13    |  N   | Gagnante
   14    |  N   | Gagnante
   15    |  N   | Gagnante
   16    |  P   | Perdante (n % 4 == 0)


**Observation** : Les P-positions sont exactement les multiples de 4 !

---

## 2. Le jeu de Nim

**Nim** est le jeu combinatoire le plus celebre :

- Plusieurs **tas** de jetons
- A chaque tour, retirer **au moins un jeton** d'un **seul tas**
- Le joueur qui retire le **dernier jeton** gagne

### Theoreme de Bouton (1901)

Une position de Nim avec des tas de tailles n1, n2, ..., nk est une :
- **P-position** si n1 XOR n2 XOR ... XOR nk = 0
- **N-position** sinon

In [3]:
def nim_sum(*heaps) -> int:
    """Calcule la nim-sum (XOR) des tailles de tas."""
    result = 0
    for h in heaps:
        result ^= h
    return result

def nim_position_type(*heaps) -> str:
    """Determine si une position est P ou N."""
    return 'P' if nim_sum(*heaps) == 0 else 'N'

# Exemples
examples = [
    (1, 2, 3),
    (1, 2, 4),
    (3, 5, 6),
    (7, 7),
    (1, 1, 1),
]

print("Tas           | Nim-sum | Type")
print("-" * 35)
for heaps in examples:
    ns = nim_sum(*heaps)
    t = nim_position_type(*heaps)
    print(f"{str(heaps):13} | {ns:7} | {t}")

Tas           | Nim-sum | Type
-----------------------------------
(1, 2, 3)     |       0 | P
(1, 2, 4)     |       7 | N
(3, 5, 6)     |       0 | P
(7, 7)        |       0 | P
(1, 1, 1)     |       1 | N


### Strategie gagnante

Si vous etes en N-position (nim-sum != 0), vous pouvez toujours jouer pour atteindre une P-position.

In [4]:
def find_winning_move(heaps: list) -> tuple:
    """
    Trouve un coup gagnant au Nim.
    Retourne (index_tas, nouvelle_taille) ou None si P-position.
    """
    ns = nim_sum(*heaps)
    if ns == 0:
        return None  # P-position, pas de coup gagnant
    
    for i, h in enumerate(heaps):
        # Nouvelle taille pour ce tas
        new_h = h ^ ns
        if new_h < h:  # On doit retirer, pas ajouter
            return (i, new_h)
    
    return None

# Exemple
heaps = [3, 5, 7]
print(f"Position: {heaps}")
print(f"Nim-sum: {nim_sum(*heaps)}")

move = find_winning_move(heaps)
if move:
    i, new_h = move
    print(f"\nCoup gagnant: reduire tas {i} de {heaps[i]} a {new_h}")
    new_heaps = heaps.copy()
    new_heaps[i] = new_h
    print(f"Nouvelle position: {new_heaps}")
    print(f"Nouvelle nim-sum: {nim_sum(*new_heaps)}")

Position: [3, 5, 7]
Nim-sum: 1

Coup gagnant: reduire tas 0 de 3 a 2
Nouvelle position: [2, 5, 7]
Nouvelle nim-sum: 0


---

## 3. Fonction Mex et valeur de Grundy

### Fonction Mex (Minimum Excludant)

Le **mex** d'un ensemble S est le plus petit entier naturel **absent** de S.

In [5]:
def mex(s: set) -> int:
    """Minimum excludant : plus petit entier >= 0 absent de l'ensemble."""
    n = 0
    while n in s:
        n += 1
    return n

# Exemples
examples = [
    set(),           # mex = 0
    {0},             # mex = 1
    {0, 1, 2},       # mex = 3
    {1, 2, 3},       # mex = 0 (0 absent)
    {0, 2, 4},       # mex = 1
]

for s in examples:
    print(f"mex({s}) = {mex(s)}")

mex(set()) = 0
mex({0}) = 1
mex({0, 1, 2}) = 3
mex({1, 2, 3}) = 0
mex({0, 2, 4}) = 1


### Valeur de Grundy

La **valeur de Grundy** (ou nimber) d'une position G est definie recursivement :

- Position terminale : Grundy = 0
- Sinon : Grundy(G) = mex{Grundy(G') : G' accessible depuis G}

Propriete importante :
- P-position ssi Grundy = 0
- N-position ssi Grundy > 0

In [6]:
def grundy_nim(n: int, memo: dict = None) -> int:
    """
    Valeur de Grundy de nim(n) - un seul tas de n jetons.
    (Spoiler: c'est toujours n)
    """
    if memo is None:
        memo = {}
    if n in memo:
        return memo[n]
    
    if n == 0:
        return 0
    
    # Positions accessibles : nim(0), nim(1), ..., nim(n-1)
    reachable = {grundy_nim(i, memo) for i in range(n)}
    result = mex(reachable)
    memo[n] = result
    return result

print("n  | grundy(nim(n)) | Verification")
print("-" * 35)
for n in range(10):
    g = grundy_nim(n)
    verify = "OK" if g == n else "ERREUR"
    print(f"{n:2} | {g:14} | {verify}")

n  | grundy(nim(n)) | Verification
-----------------------------------
 0 |              0 | OK
 1 |              1 | OK
 2 |              2 | OK
 3 |              3 | OK
 4 |              4 | OK
 5 |              5 | OK
 6 |              6 | OK
 7 |              7 | OK
 8 |              8 | OK
 9 |              9 | OK


---

## 4. Theoreme de Sprague-Grundy

Le theoreme de Sprague-Grundy est le resultat central de la theorie des jeux combinatoires :

> **Theoreme** : Tout jeu impartial est equivalent a un tas de Nim.
> 
> Pour une **somme de jeux** G1 + G2 + ... + Gk :
> Grundy(G1 + G2 + ... + Gk) = Grundy(G1) XOR Grundy(G2) XOR ... XOR Grundy(Gk)

Cela signifie que pour analyser n'importe quel jeu impartial, il suffit de :
1. Calculer la valeur de Grundy de chaque composante
2. XOR les valeurs ensemble
3. Si le resultat est 0, c'est une P-position, sinon N-position

In [7]:
def analyze_nim_game(heaps: list) -> dict:
    """
    Analyse complete d'une position de Nim.
    """
    grundy_values = heaps  # Pour Nim, grundy(n) = n
    total_grundy = nim_sum(*grundy_values)
    
    result = {
        'heaps': heaps,
        'grundy_values': grundy_values,
        'nim_sum': total_grundy,
        'position_type': 'P' if total_grundy == 0 else 'N',
        'winning_move': find_winning_move(heaps)
    }
    return result

# Exemple
game = analyze_nim_game([4, 7, 9])

print(f"Position: {game['heaps']}")
print(f"Valeurs de Grundy: {game['grundy_values']}")
print(f"Nim-sum: {game['nim_sum']}")
print(f"Type: {game['position_type']}-position")

if game['winning_move']:
    i, new_h = game['winning_move']
    print(f"\nCoup gagnant: tas {i}: {game['heaps'][i]} -> {new_h}")

Position: [4, 7, 9]
Valeurs de Grundy: [4, 7, 9]
Nim-sum: 10
Type: N-position

Coup gagnant: tas 2: 9 -> 3


---

## 5. Jeux de soustraction

Un **jeu de soustraction** S(M) est defini par un ensemble de coups M :
- Un tas de n jetons
- On peut retirer m jetons si m est dans M et m <= n

Par exemple, S({1, 3, 4}) : on peut retirer 1, 3 ou 4 jetons.

In [8]:
def grundy_subtraction(n: int, moves: set, memo: dict = None) -> int:
    """
    Valeur de Grundy pour un jeu de soustraction S(moves).
    """
    if memo is None:
        memo = {}
    if n in memo:
        return memo[n]
    
    if n == 0:
        return 0
    
    # Positions accessibles
    reachable = set()
    for m in moves:
        if m <= n:
            reachable.add(grundy_subtraction(n - m, moves, memo))
    
    result = mex(reachable)
    memo[n] = result
    return result

# Analyser S({1, 3, 4})
moves = {1, 3, 4}
memo = {}

print(f"Jeu de soustraction S({moves})")
print("\n n | Grundy | Type")
print("-" * 20)
for n in range(20):
    g = grundy_subtraction(n, moves, memo)
    t = 'P' if g == 0 else 'N'
    print(f"{n:2} | {g:6} | {t}")

Jeu de soustraction S({1, 3, 4})

 n | Grundy | Type
--------------------
 0 |      0 | P
 1 |      1 | N
 2 |      0 | P
 3 |      1 | N
 4 |      2 | N
 5 |      3 | N
 6 |      2 | N
 7 |      0 | P
 8 |      1 | N
 9 |      0 | P
10 |      1 | N
11 |      2 | N
12 |      3 | N
13 |      2 | N
14 |      0 | P
15 |      1 | N
16 |      0 | P
17 |      1 | N
18 |      2 | N
19 |      3 | N


**Observation** : Les valeurs de Grundy pour les jeux de soustraction sont souvent **periodiques** apres un certain point.

---

## Exercices

### Exercice 1 : Jeu de soustraction personnalise

Analysez le jeu S({1, 2, 5}) et trouvez la periode des valeurs de Grundy.

### Exercice 2 : Nim a 4 tas

Trouvez un coup gagnant depuis la position (3, 5, 7, 11).

### Exercice 3 : Jeu de Wythoff

Le **jeu de Wythoff** : deux tas, on peut soit retirer de l'un, soit retirer le meme nombre des deux. Implementez le calcul des valeurs de Grundy.

---

## Resume

| Concept | Description |
|---------|-------------|
| **P-position** | Position perdante pour le joueur a jouer |
| **N-position** | Position gagnante pour le joueur a jouer |
| **Mex** | Plus petit entier absent d'un ensemble |
| **Valeur de Grundy** | Mex des Grundy des positions accessibles |
| **Sprague-Grundy** | Grundy(G1 + G2) = Grundy(G1) XOR Grundy(G2) |
| **Nim-sum** | XOR des tailles des tas |

### Prochaines etapes

- **Side track 8b** : Formalisation en Lean 4 (PGame, mathlib)
- **Side track 8c** : Approfondissements Python (visualisations, jeux plus complexes)
- **Notebook 9** : Induction arriere pour les jeux extensifs

---

**Navigation** : [<- ExtensiveForm](GameTheory-7-ExtensiveForm.ipynb) | [Index](README.md) | [Suivant ->](GameTheory-9-BackwardInduction.ipynb)